In [2]:
import cv2
import torch
import numpy as np
from keras.models import load_model # type: ignore
from keras.applications.imagenet_utils import preprocess_input # type: ignore

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import cv2
from ultralytics import YOLO

model = YOLO("best.pt")  # Replace with actual path to your trained model


In [3]:
# from ultralytics import YOLO

# yolo_model = YOLO('best.pt')


In [4]:
# Define your traffic sign class names (adjust according to your dataset)
class_names = [
    "Speed limit 20", "Speed limit 30", "Speed limit 50", "Speed limit 60",
    "Speed limit 70", "Speed limit 80", "End of speed limit 80", "Speed limit 100",
    "Speed limit 120", "No passing", "No passing for trucks", "Right-of-way at intersection",
    "Priority road", "Yield", "Stop", "No vehicles", "Vehicles over 3.5t prohibited",
    "No entry", "General caution", "Dangerous curve left", "Dangerous curve right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right",
    "Road work", "Traffic signals", "Pedestrians", "Children crossing",
    "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing",
    "End of all restrictions", "Turn right ahead", "Turn left ahead",
    "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout", "End of no passing", "End of no passing for trucks"
]

def preprocess_cnn(image):
    image = cv2.resize(image, (32, 32))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [7]:

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1360)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 800)

frame_count = 0
cls_id = 99
prev_id = 99

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    result = results[0]

    boxes = result.boxes

    try:
        with open("NewData/labels.txt", "r") as f:
            lines = f.readlines()
            cls_id = int(lines[0])
            if cls_id != prev_id:
                frame_count = 0
                prev_id = cls_id
    except:
        cls_id = 99
        


    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cropped_img = frame[y1:y2, x1:x2]

        

        # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
        img_path = f"NewData/images/{cls_id:02}{frame_count:03d}.jpg"
        cv2.imwrite(img_path, frame)

        x_center = (x1 + x2) / 2 / frame.shape[1]
        y_center = (y1 + y2) / 2 / frame.shape[0]
        width = (x2 - x1) / frame.shape[1]
        height = (y2 - y1) / frame.shape[0]
        label_path = f"NewData/labels/{cls_id:02}{frame_count:03d}.txt"
        with open(label_path, "w") as f:
            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


        print(f"\rframe_count: {frame_count}", end="", flush=True)
        frame_count += 1

    cv2.imshow(f"Traffic Sign Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 53.4ms
Speed: 3.5ms preprocess, 53.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.8ms
Speed: 3.2ms preprocess, 81.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 109.7ms
Speed: 2.4ms preprocess, 109.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 82.9ms
Speed: 2.2ms preprocess, 82.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 69.2ms
Speed: 2.4ms preprocess, 69.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 69.1ms
Speed: 2.2ms preprocess, 69.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.1ms
Speed: 2.2ms preprocess, 65.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.1ms
Speed: 1.9ms preprocess, 64.1ms

In [ ]:
# import os

# # Create the directory if it doesn't exist
# os.makedirs("NewData/images", exist_ok=True)

# frame_count = 0

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Save the frame as an image
#     img_path = f"NewData/images/frame_{frame_count:05d}.jpg"
#     cv2.imwrite(img_path, frame)
#     # Save bounding box coordinates if any boxes are detected
#     results = model(frame)
#     result = results[0]
#     boxes = result.boxes

#     for box in boxes:
#         x1, y1, x2, y2 = map(int, box.xyxy[0])
#         cls_id = int(box.cls[0])
#         conf = float(box.conf[0])
#         # Save in YOLO format: <class> <x_center> <y_center> <width> <height> <confidence>
#         x_center = (x1 + x2) / 2 / frame.shape[1]
#         y_center = (y1 + y2) / 2 / frame.shape[0]
#         width = (x2 - x1) / frame.shape[1]
#         height = (y2 - y1) / frame.shape[0]
#         label_path = f"NewData/images/frame_{frame_count:05d}.txt"
#         with open(label_path, "w") as f:
#             f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {conf:.4f}\n")

#     frame_count += 1

#     # Display the frame (optional)
#     cv2.imshow("Frame", frame)
#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
# try:
#     with open("NewData/labels.txt", "r") as f:
#         lines = f.readlines()
#         cls_id = int(lines[0])
# except:
#     cls_id = 99

# print(f"Class ID: {cls_id}")

Class ID: 3
